In [7]:
from importlib import reload
import sentence_generation
reload(sentence_generation)
from sentence_generation import *

In [1]:
import os
os.sys.path.insert(0, '../script')
from webnlg import WebNLGCorpus
from evaluation import *
from lexicalization import *
from sentence_generation import *
from content_selection import SelectAllContentSelection
from collections import defaultdict, Counter
from template_extraction import *
from discourse_structuring import *
from sentence_aggregation import *
from text_generation import TemplateBasedTextGenerator
from data_alignment import *
import re

from textacy import similarity

In [30]:
v_12 = WebNLGCorpus.load(['dev_1.2', 'train_1.2'])
v_12_ntriples_1 = v_12.subset(ntriples=1)

# content selection
cs = SelectAllContentSelection()

# sentence generation
t = ManualTemplateExtract()
template_db = t.extract(v_12_ntriples_1)

mfe = MostFrequentTemplateSentenceGenerator()
mfe.fit(template_db)

npsg = NearestPredicateTemplateSentenceGenerator(sentence_generator=mfe,
                                                 similarity_metric=similarity.levenshtein,
                                                 threshold=0.5)
npsg.fit(template_db)

fsg = FallBackPipelineSentenceGenerator([mfe, npsg, JustJoinTripleSentenceGenerator()])


# lexicalization
la = LexicalizeAsAligned()
la.fit(v_12_ntriples_1)

# discourse structuring
ds = ChainDiscourseStructuring()

# sentence aggregation
sa = OneSentenceAggregator()

nlg = TemplateBasedTextGenerator(cs, fsg, la, ds, sa)

In [31]:
evaluate_model(nlg, 'ola')

{'bleu': 44.01, 'meteor': 0.3909775015368552, 'ter': 0.5066825729733884}

In [32]:
template_db['creator']

Counter({{subject} was created by {object} .: 58,
         The creator of {subject} is {object} .: 13,
         {subject} , was created by {object} .: 3,
         {object} created {subject} .: 8,
         {object} is one of the creators of {subject} .: 1,
         {object} is the creator of {subject} .: 13,
         {subject} were created by {object} .: 1,
         {subject} is created by {object} .: 2,
         {subject} is a fictional character created by {object} .: 1,
         {object} was the creator of {subject} .: 3,
         {object} was one of the creators of {subject} .: 1,
         {subject} was created by {object} in the comics genre .: 1,
         {subject} is a comic character created by {object} .: 1,
         {subject} , The Arrow , was created by {object} .: 1,
         {subject} was created at {object} .: 1,
         {subject} was created in {object} .: 1})

# Examples

In [4]:
test = WebNLGCorpus.load(['test_with_lex'])

## Example 1

In [28]:
sample = test.sample(ntriples=3)

text = nlg.predict_entry(sample.get_data())

sample

Triple info: category=Building eid=Id471

	Modified triples:

Alan_B._Miller_Hall | buildingStartDate | "30 March 2007"
Mason_School_of_Business | country | United_States
Alan_B._Miller_Hall | currentTenants | Mason_School_of_Business


	Lexicalizations:

Alan B. Miller Hall was started on March 30,2007 and has The Mason School of Business in the U.S. as a tenant. || 
Alan B. Miller Hall's building opened in 30th March 2007. The Mason School of Business in the United States are the current tenants of Alan B Miller Hall. || 

In [29]:
text

'The construction of alan b miller hall began in 30th march 2007 . the mason school of business is in the united states . the mason school of business are the current tenants of alan b miller hall .'

## Example 2

In [30]:
sample = test.sample()

text = nlg.predict_entry(sample.get_data())

sample

Triple info: category=SportsTeam eid=Id125

	Modified triples:

Michele_Marcolini | club | Atalanta_B.C.


	Lexicalizations:

Michele Marcolini has played for Atalanta BC. || 
Michele Marcolini is part of the Atalanta B.C. club. || 
Michele Marcolini plays for Atalanta B.C. || 

In [31]:
text

'michele marcolini plays for Atalanta B.C. .'